#Predicting the sale price of bulldozers using machine learning

In this notebook ,we're going to thorigh an example machine learning project with the goal of predicting the sale price of bulldozers.

## 1.Problem definition

> How well can we predict the future sale price of a bulldozer,given its characteristsics and previous examples of how much similar bulldozers have been sold for?

## 2. Data

The data is downloaded from the Kaggle Blue Book for bulldozers 
competition:https://www.kaggle.com/c/bluebook-for-bulldozers/data

There three main datasets:

* Train.csv is the training set, which contains data through the end of 2011.
* Valid.csv is the validation set, which contains data from January 1, 2012 - April 30, 2012 You make predictions on this set throughout the majority of the competition. Your score on this set is used to create the public leaderboard.
* Test.csv is the test set, which won't be released until the last week of the competition. It contains data from May 1, 2012 - November 2012. Your score on the test set determines your final rank for the competition.

## 3. Evaluation

The evaluation metric for this competition is the RMSLE (root mean squared log error) between the actual and predicted auction prices.

For more on the evaluation of this project check:
https://www.kaggle.com/c/bluebook-for-bulldozers/overview/evaluation

Note: The goal for most regression evaluation metrics  is to minimize the error.For example ,our goal for this project will be to build a machine learning model which minimises RMSLE.

## 4. Features

Kaggle provides a data dictionary detailing all of the features of the dataset.You can view this data dictionary here:

https://www.kaggle.com/c/bluebook-for-bulldozers/download/Data%20Dictionary.xlsx





In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [ ]:
#Import training and validation sets
df=pd.read_csv('../input/bluebook-for-bulldozers/TrainAndValid.csv',
              low_memory=False)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
fig,ax=plt.subplots(figsize=(10,5))
ax.scatter(df["state"][:1000],df["SalePrice"][:1000]);

In [ ]:
#getting the first 10000 sale date to view format 
df.saledate[:1000]

In [ ]:
#Checking the distribution of our target variable through a histogram plot
df.SalePrice.plot.hist();

Most of the sale prices are between 200000-600000

### Parsing dates

When we work with time series data, we want to enrich the time  & date  component as much as possible.

We can do that by telling pandas which of our columns has dates in it using the parse_dates parameter.

In [ ]:
#Import data again bu this time parse dates
df=pd.read_csv('../input/bluebook-for-bulldozers/TrainAndValid.csv',
              low_memory=False,
              parse_dates=["saledate"])

In [ ]:
#Getting the data type of our Saledate column after parsing dates
df.saledate.dtype

**Tip from the Sale Date data type**

datetime64[ns] is a general dtype, while <M8[ns] is a specific dtype. 
General dtypes map to specific dtypes, but may be different from one installation of NumPy to the next.

source: stakeoverflow : https://stackoverflow.com/questions/29206612/difference-between-data-type-datetime64ns-and-m8ns

In [ ]:
#Note the difference now in Saledate format after parse_dates
df.saledate[:1000]

In [ ]:
#plotting bulldozer sale dates in relation to their Sale Prices
fig,ax=plt.subplots()
ax.scatter(df["saledate"][:1000],df["SalePrice"][:1000]);


In [ ]:
#Year with majority of sales
df.saledate.value_counts().sort_values(ascending=False)

February is the month with the most sales from 2008-2012 with ,february 2009 having the highest sales.

In [ ]:
df.head()

### Sort DataFrame by saledate

When working with time series data,it's a good idea to sort it by date.

In [ ]:
#sort DataFrame in date order
df.sort_values(by=["saledate"],inplace=True,ascending=True)
df.saledate.head(20)

### Make copy of the original DataFrame
We make a copy of the original dataframe so when we manipulate the copy ,we've still got our original data.

In [ ]:
#make a copy
df_tmp=df.copy()

In [ ]:
### sort bulldozer  Year make in ascending order
df.sort_values(by=["YearMade"],ascending=True)

In [ ]:
### Sorting bulldozer prices in ascending
df.sort_values(by=["SalePrice","YearMade"],ascending=True)

### Add dataframe parameters for saledate column

In [ ]:
#this part of feature engineering
df_tmp["saleYear"]=df_tmp.saledate.dt.year
df_tmp["saleMonth"]=df_tmp.saledate.dt.month
df_tmp["saleDay"]=df_tmp.saledate.dt.day
df_tmp["saleDayOfWeek"]=df_tmp.saledate.dt.dayofweek
df_tmp["saleDayofYear"]=df_tmp.saledate.dt.dayofyear


In [ ]:
df_tmp.head().T

In [ ]:
#Now we've enriched our DataFrame with datetime features we can remove saledate column
df_tmp.drop("saledate",axis=1,inplace=True)

In [ ]:
#Check the values of different columns
df_tmp.state.value_counts()

In [ ]:
df_tmp.head()

## 5.Modelling

we've done enough EDA(we could always do more)but let's start to do some model-driven EDA.

In [ ]:
#Let's build a machine learning model.
from sklearn.ensemble import RandomForestRegressor

model=RandomForestRegressor(n_jobs=1,
                           random_state=42)

#model.fit(df_tmp.drop("SalePrice",axis=1),df_tmp["SalePrice"])

In [ ]:
df_tmp.info()

In [ ]:
df_tmp.isna().sum()

### Convert strings to categories

One way we can turn all of our data into numbers is by converting them into pandas categories.

In [ ]:
pd.api.types.is_string_dtype(df_tmp["UsageBand"])

In [ ]:
#Find the columns which contains strings
for label,content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        print(label)
    

In [ ]:
#This will turn all the string values into category values.
for label,content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label]=content.astype("category").cat.as_ordered()

In [ ]:
df_tmp.dtypes

In [ ]:
df_tmp.state.cat.categories

In [ ]:
df_tmp.state.cat.codes

Thanks to pandas Categories we now have a way to access all of our data in the form of numbers.

More on this can be found here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Categorical.html

But we still have a bunch of missing data.

In [ ]:
#Check for mssing data
df_tmp.isna().sum()/len(df_tmp)

### Save preprocessed data

In [ ]:
#Export current tmp dataframe localy to pc
df_tmp.to_csv(r"D:\DS\bulldozer-price-prediction-project\data\bluebook-for-bulldozers\train_temp.csv",index=False)


In [ ]:
#Import preprocessed data from pc
df_tmp=pd.read_csv(r"D:\DS\bulldozer-price-prediction-project\data\bluebook-for-bulldozers\train_temp.csv",
             low_memory=False)

In [ ]:
df_tmp.head().T

In [ ]:
df_tmp.isna().sum()

## Fill missing values

#### Fill numeric missing values First

In [ ]:
#check for numeric data types from our data set
for label,content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        print (label)

In [ ]:
#Check for which numeric columns have null values
for label,content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

In [ ]:
#Fill numeric rows with the median
for label,content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            #Add a binary column which tells us if the data was missing or not
            df_tmp[label+"_is_missing"]=pd.isnull(content)
            #Fill missing numeric values with median
            df_tmp[label]=content.fillna(content.median())

median is more robust than mean to outliers.

In [ ]:
#Check if there's any numeric values
for label,content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

In [ ]:
df_tmp.isna().sum()

In [ ]:
#Check to see how many examples were missing
df_tmp.auctioneerID_is_missing.value_counts()

### Filling and turning categorical variables into numbers

In [ ]:
#check for columns which aren't numeric
for label,content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)

In [ ]:
#Turn categorical values into number and fill missing
for label,content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        #Add binary column to indicate whether sample had missing  value or not
        df_tmp[label+"_is_missing"]=pd.isnull(content)
        #Turn categories into numbers and add +1(we add +1 to give missing values 0 instead of -1)
        df_tmp[label]=pd.Categorical(content).codes+1

In [ ]:
df_tmp.head().T

In [ ]:
pd.Categorical(df_tmp["state"]).codes+1

Now that all of our data is numeric as well our data frame has no missing values we should be able to build a machine learning model.

In [ ]:
df_tmp.info()

# Building a machine learning model.

In [ ]:
%%time 
#this calculates how much time this row gets to execute the code

#instantiate model
model=RandomForestRegressor(n_jobs=-1,
                           random_state=42)

#fit the model
model.fit(df_tmp.drop("SalePrice",axis=1),df_tmp["SalePrice"])

In [ ]:
#Score the model
model.score(df_tmp.drop("SalePrice",axis=1),df_tmp["SalePrice"])

The data was tested on data it already knew about training data instead of test data (data that it's not seen) .This is why this metric is not reliable.

## Splitting train/validation sets

In [ ]:
df_tmp.saleYear.value_counts()

In [ ]:
#Split data into training and validation
df_val=df_tmp[df_tmp.saleYear==2012]
df_train=df_tmp[df_tmp.saleYear != 2012]

len(df_val),len(df_train)

In [ ]:

# Split data into X & y
X_train, y_train = df_train.drop("SalePrice", axis=1), df_train.SalePrice
X_valid, y_valid = df_val.drop("SalePrice", axis=1), df_val.SalePrice

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

### Building an evaluation function

In [ ]:
#Create  evaluation function (the competion uses RMSLE)
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score

def rmsle(y_test, y_preds):
    """
    Caculates root mean squared log error between predictions and
    true labels.
    """
    return np.sqrt(mean_squared_log_error(y_test, y_preds))

# Create function to evaluate model on a few different levels
def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {"Training MAE": mean_absolute_error(y_train, train_preds),
              "Valid MAE": mean_absolute_error(y_valid, val_preds),
              "Training RMSLE": rmsle(y_train, train_preds),
              "Valid RMSLE": rmsle(y_valid, val_preds),
              "Training R^2": r2_score(y_train, train_preds),
              "Valid R^2": r2_score(y_valid, val_preds)}
    return scores

## Testing our model on a subset (to tune the hyperparameters)

In [ ]:
#Change max_samples value in our Random Forest model
model=RandomForestRegressor(n_jobs=-1,
                           random_state=42,
                           max_samples=10000)
model

In [ ]:
%%time 
#cutting down on the max number of samples each estimator can see improves training time
#currently using 10,0000 instead of 401125
model.fit(X_train,y_train)

In [ ]:
show_scores(model)

# Hyperparameter tuning with RandomizedSearchCV

In [ ]:
%%time
from sklearn.model_selection import RandomizedSearchCV

# Different RandomForestRegressor hyperparameters
rf_grid = {"n_estimators": np.arange(10, 100, 10),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2),
           "max_features": [0.5, 1, "sqrt", "auto"],
           "max_samples": [10000]}

# Instantiate RandomizedSearchCV model
rs_model = RandomizedSearchCV(RandomForestRegressor(n_jobs=-1,
                                                    random_state=42),
                              param_distributions=rf_grid,
                              n_iter=5,
                              cv=5,
                              verbose=True)

# Fit the RandomizedSearchCV model
rs_model.fit(X_train, y_train)

In [ ]:
#Find the best model hyperparameters
rs_model.best_params_

In [ ]:
#Evaluate the RandomizedSearch model
show_scores(rs_model)

## Train a model with the best hyperparameter

**Note** These were found after 100 iterations of RandomizedSearchCV

In [ ]:
%%time
#Most ideal hyperparameters
ideal_model=RandomForestRegressor(n_estimators=40,
                                 min_samples_leaf=1,
                                 min_samples_split=14,
                                 max_features=0.5,
                                 n_jobs=-1,
                                 max_samples=None,
                                 random_state=42) #random state so that our results are reproducible
#Fit the ideal model
ideal_model.fit(X_train,y_train)

In [ ]:
#scores for ideal model (trained on all the data)
show_scores(ideal_model)

In [ ]:
#scores on rs_model(only trained on -10,000 examples)
show_scores(rs_model)

## Make predictions on test data

In [ ]:
#import the test data
df_test=pd.read_csv('../input/bluebook-for-bulldozers/Test.csv',
                   low_memory=False,
                   parse_dates=["saledate"])

df_test.head()

# Preprocessing test data

In [ ]:
def preprocess_data(df):
    """
    Performs transformations on df and returns transformed df."""
    df["saleYear"]=df.saledate.dt.year
    df["saleMonth"]=df.saledate.dt.month
    df["saleDay"]=df.saledate.dt.day
    df["saleDayOfWeek"]=df.saledate.dt.dayofweek
    df["saleDayofYear"]=df.saledate.dt.dayofyear

    df.drop("saledate",axis=1, inplace=True)
    #Fill the numeric rows with median
    for label,content in df.items():
        if pd.api.types.is_numeric_dtype(content):
            if pd.isnull(content).sum():
                #Add a binary column which tells us if the data was missing or not
                df[label+"_is_missing"]=pd.isnull(content)
                #Fill missing numeric values with median
                df[label]=content.fillna(content.median())
    
        #Filled categorical missing data and turn categories into numbers
    
        if not pd.api.types.is_numeric_dtype(content):
            df[label+"_is_missing"]=pd.isnull(content)
                #We add +1 to the category code because pandas encodes missing categories as -1
            df[label]=pd.Categorical(content).codes+1
    return df

In [ ]:
#Process test data
df_test=preprocess_data(df_test)
df_test.head()

In [ ]:
#We can find how the columns differ using sets
set(X_train.columns)-set(df_test.columns)

In [ ]:
#Manually adjust df_test to have auctioneerID_is_missing column
df_test["auctioneerID_is_missing"]=False
df_test.head()

Finally now our test dataframe has the same features as our training dataframe.We can now make predictions!

# Make predictions using test data

In [ ]:
#Make predictions on updated test data
test_preds=ideal_model.predict(df_test)

In [ ]:
test_preds

We've have made the predictions but they are not in the same format kaggle is asking for:
https://www.kaggle.com/c/bluebook-for-bulldozers/overview/evaluation

In [ ]:
#Format predictions into the same format Kaggle is after
df_preds=pd.DataFrame()
df_preds["SalesID"]=df_test["SalesID"]
df_preds["SalesPrice"]=test_preds
df_preds

In [ ]:
#Export prediction data
df_preds.to_csv(r"D:\DS\bulldozer-price-prediction-project\data\bluebook-for-bulldozers\test_predictions.csv",index=False)

# Feature Importance

Feature importance seeks to figure out which different attributes of the data were most important when it comes to predicting **target variable**(SalePrice).


In [ ]:
#Find feature importance of our best model
ideal_model.feature_importances_

In [ ]:
#Helper function for plotting feature importance
def plot_features(columns, importances, n=20):
    df = (pd.DataFrame({"features": columns,
                        "feature_importances": importances})
          .sort_values("feature_importances", ascending=False)
          .reset_index(drop=True))
    
    # Plot the dataframe
    fig, ax = plt.subplots()
    ax.barh(df["features"][:n], df["feature_importances"][:20])
    ax.set_ylabel("Features")
    ax.set_xlabel("Feature importance")
    ax.invert_yaxis()


In [ ]:
plot_features(X_train.columns, ideal_model.feature_importances_)



# XGBoost Regressor Model



In [ ]:
#using XGboost Regression model to improve our rmsle
from xgboost import XGBRegressor
#instantiate our xgboost model
xgbr_model=XGBRegressor(n_estimators=1000,
                      learning_rate=0.05,
                      max_depth=8,
                      random_state=42)

#using the model to fit on our training data
xgbr_model.fit(X_train,y_train)
xgbr_preds=xgbr_model.predict(X_valid)

In [ ]:
xgbr_score=r2_score(y_valid,xgbr_preds)
xgbr_score

In [ ]:
#getting the rmsle as its the evaluation metric 
np.sqrt(mean_squared_log_error(y_valid,xgbr_preds))

In [ ]:
xgbr_test_preds=xgbr_model.predict(df_test)

In [ ]:
xgbr_test_preds

In [ ]:
#Format predictions into the same format Kaggle is after
df_preds=pd.DataFrame()
df_preds["SalesID"]=df_test["SalesID"]
df_preds["SalesPrice"]=xgbr_test_preds
df_preds

In [ ]:
#Export prediction data to local pc
df_preds.to_csv(r"D:\DS\bulldozer-price-prediction-project\data\bluebook-for-bulldozers\test_predictions.csv",index=False)

In [ ]:
xgbr_model.feature_importances_

In [ ]:
plot_features(X_train.columns, xgbr_model.feature_importances_)

The above graph shows the important features worth concidering for bulldozer house predictions based on test data.

#  Conclusion
Using XGBoost Regressor algorithm gave a slight improvement to our ideal Random Forest Regressor model from the evaluation metric used.